# Try 01.
- 네이버 카페 크롤러 참고했습니당

In [1]:
#Step 0. 필요한 모듈과 라이브러리 로딩
from bs4 import BeautifulSoup     
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import random
import pandas as pd
import json
from datetime import datetime

In [2]:
def crawling():
     for content_num in range(4, 24):
        try:
           
            #--- 제목 수집
            time.sleep(random.uniform(0.5,1.2))
            title = [driver.find_element(By.CLASS_NAME,'title_subject').text]
            #titles.append(title)

            #--- 날짜 수집
            time.sleep(random.uniform(0.2, 1.3))
            date = driver.find_element(By.CLASS_NAME, "gall_date").text
            dates.append(date)

            #--- 본문 수집
            time.sleep(random.uniform(0.5, 1.5))
            content = driver.find_element(By.CLASS_NAME, "write_div").text
            content = [content.replace("\n", ' ')]   # '\n'을 공백으로
            #contents.append(content)

            #--- 댓글 수집
            soup = BeautifulSoup(driver.page_source, 'lxml') 
            iscomment = soup.find_all(class_='usertxt ub-word')
            if len(iscomment) == 0: 
                box = []
            else:
                WebDriverWait(driver, 15).until( EC.presence_of_all_elements_located((By.CLASS_NAME, "comment_box")) )
                soup = BeautifulSoup(driver.page_source, 'lxml')
                iscomment = soup.find_all(class_='usertxt ub-word')
                box = []
                for i in iscomment:
                    box.append(i.get_text())


            contents.append(title + content + box)
            
                        
            # 키워드 만들기
            keywords.append(keyword)

            # 다음 게시물 클릭 및 반복
            driver.find_element(By.CSS_SELECTOR, '#bottom_listwrap > section.left_content.result > article > div.gall_listwrap.list > table > tbody > tr:nth-child({0}) > td.gall_tit.ub-word > a.search_list'.format(content_num)).click()
            driver.implicitly_wait(3)
            
        except:
            break

In [3]:
# Step 4. 제목, 날짜, 본문, 댓글 리스트 만들고 수집을 위해 제목 클릭해서 들어가기
# 제목, 날짜, 본문, 댓글(box) 빈 list 생성
titles = []
dates = []
contents = []
comments = []
keywords = []

In [4]:
# ----------------------- 크롤링 키워드 ---------------------------
query_txt = ['음쓰', '유지비', '플스' '피규어+레고', '담배냄새', '요리', '쇠질', '취미', '아침밥', '도시락', '빨래','집청소','주차']

#Step 1. 크롬 드라이버를 사용해서 웹 브라우저 실행

path = "C:/Users/user/chromedriver.exe"
driver = webdriver.Chrome(path)
url = "https://gall.dcinside.com/board/lists?id="
gallery = 'room'

'''driver.get(url)   ###+gallery)
time.sleep(random.randrange(1,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택 '''
    
for keyword in query_txt:
    
    driver.get(url+gallery)   ###+gallery)
    time.sleep(random.randrange(1,5)) 
    
    #Step 2. 검색창의 이름을 찾아서 검색어 입력

    element = driver.find_element(By.NAME, "search_keyword")
    # element.send_keys(Keys.CONTROL + "A") # Keys.LeftShift + Keys.Home
    element.send_keys(keyword)
    element.find_element(By.XPATH, '/html/body/div[2]/div[3]/main/section[1]/article[2]/form[2]/fieldset/div/div[2]/button').click()
    
    # Step 3. 페이지에서 가져올 컨텐츠 찾기
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    try:
            
        # Step 5. 첫번째 게시물 클릭 (선택자 입력_여기서는 xpath copy 해줌) 및 데이터 수집 시작

        driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/main/section[1]/article[2]/div[3]/table/tbody/tr[3]/td[2]/a[1]').click()
        crawling()
        
        
        # Step 6. 마지막 페이지 숫자 찾기

        try : # 15페이지가 넘는 경우
            driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/main/section/article[3]/div/section[1]/article/div[3]/div[1]/a[16]').click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            div = soup.find('div', {'class':'bottom_paging_wrap re'})
            em = div.find('em')
            last = em.text

            page = div.find('a')
            link = 'https://gall.dcinside.com'+ page.attrs['href']

        except : # 15페이지보다 적은 경우
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            div = soup.find('div', {'class': 'bottom_paging_wrapre'})
            a_tag = div.find_all('a')
            last = a_tag[-2].text

            page = div.find('a')
            link = 'https://gall.dcinside.com'+ page.attrs['href']


        # Step 7. 이동할 페이지 링크 찾기

        links = []
        for new_page_number in range(2,int(last)+1):
            page_number = link.split('page=')[1].split('&')[0]
            new_link = link.replace(f"page={page_number}", f"page={new_page_number}")
            links.append(new_link)

        driver.quit()

        # Step 8. 다음 페이지들에 대해서도 데이터 수집 시작

        for link in links:
            path = "C:/Users/user/chromedriver.exe"
            driver = webdriver.Chrome(path)
            driver.get(link)
            # 첫번째 게시물 클릭 (선택자 입력_여기서는 xpath copy 해줌) 및 데이터 수집 시작
            driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/main/section[1]/article[2]/div[3]/table/tbody/tr[3]/td[2]/a[1]').click()
            crawling()
        
    except:
        pass

C:\Users\user\AppData\Local\Temp\ipykernel_23048\3955271208.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
C:\Users\user\AppData\Local\Temp\ipykernel_23048\3955271208.py:74: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [7]:
#===데이터 합치기(크롤링한 데이터를 하나의 dataframe 형태로 바꿔줌)
data = {'keyword': keywords,
        #'tit' : titles,
        'body': [','.join(map(str, content)) for content in contents],
        'date' : dates,
        }

dataDF = pd.DataFrame(data)

#csv로 저장
today = datetime.today().strftime('%Y%m%d')
dataDF.to_csv("%s_%s_20~30대남성3.csv"%(gallery, today), encoding = "utf-8-sig")

#print('============ %s페이지 수집 완료 ==============='%(link.split('page=')[1].split('&')[0]))

In [8]:
dataDF

,keyword,body,date
0,음쓰,"집안일 줄일수있는 가전 추천좀..,가족 집안일 원래는 내가 좀 했었는데 곧 다른지역...",2023.03.31 18:35:51
1,음쓰,"음쓰냉장고나 미생물처리 써보신분 잇나여,처음 자취해보는데 음식물쓰래기가 안나오는것도...",2023.03.26 13:28:16
2,음쓰,"공용 건조기랑 음쓰봉투 제공되는 집,내가 계약한 집 - dc official App",2023.03.18 09:56:22
3,음쓰,"종량제쓰레기봉투 질문좀,음쓰봉투랑 일반쓰레기봉투 둘다 서울시 내에서 이사가도 쓸 수...",2023.03.16 22:19:59
4,음쓰,"룸붕이 이사오고 첫 그리마 등장.,날씨 따뜻해지니 슬슬 원룸 단점 나오네 싱크대에...",2023.03.13 09:55:14
...,...,...,...
180,주차,"빌라주차난이도,나이트메어 나머지5대의 운명이 단한대에 달려있네 ㅋㅋ - dc of...",2022.08.31 22:42:48
181,주차,"빌라는 주차어케하냐,대부분 2중이던데 맨날 연락오면 빼고 그런거임?,연락해도 안받고...",2022.08.22 19:42:10
182,주차,"오피스텔 지하주차장에 시동키고 있는놈들 뭐냐?,꼭 층마다 한두대씩 시동 켜놓고 있음...",2022.08.22 12:04:53
183,주차,"집보고왓어,오늘도 가야되... 1기 신도시 쪽인데 신축오피 1000/65 구축오...",2022.08.05 09:58:43
